In [1]:
# simple data manipulation
import numpy  as np
import pandas as pd

# deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from   torch.utils.data import DataLoader, TensorDataset

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# results logging
import wandb

# progress bar
from   tqdm.notebook import tqdm, trange

# remove warnings (remove deprecated warnings)
import warnings
warnings.simplefilter('ignore')

# visualization of resultsa
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from   matplotlib.ticker import MaxNLocator
import seaborn           as sns

# Graph Algorithms.
import networkx as nx

# Google Colab (many lines are removed)
import os
import zipfile
# from google.colab import drive
# from distutils.dir_util import copy_tree

# wheter we are using colab or not
COLAB: bool = False
if not COLAB and not os.path.exists('./data/simulations'): 
    os.chdir('..')

# Simulation Settings
from g6smart.sim_config import SimConfig
from g6smart.evaluation import rate as rate_metrics

config = SimConfig(0)
config

Simulation Parameters: 

|                      name |                     value |
---------------------------------------------------------
|        num_of_subnetworks |                   20.0000 |
|              n_subchannel |                    4.0000 |
|             deploy_length |                   20.0000 |
|             subnet_radius |                    1.0000 |
|                      minD |                    0.8000 |
|               minDistance |                    2.0000 |
|                 bandwidth |             40000000.0000 |
|              ch_bandwidth |             10000000.0000 |
|                        fc |           6000000000.0000 |
|                    lambdA |                    0.0500 |
|                  clutType |                     dense |
|                  clutSize |                    2.0000 |
|                  clutDens |                    0.6000 |
|                   shadStd |                    7.2000 |
|                 max_power |                  

In [2]:
def setup_wandb(name: str, config: dict[str, float]):
    config['name'] = name
    wandb.init(
        project="6GSmartRRM",
        name   = name,
        config = config
    )


## Simulations and Information

Thanks to the given scripts, we can load a group of generated simulations. They don't have any solutions (neither approximations).

In [3]:
# Moung Google Drive Code
if COLAB:
    # drive.mount('/content/drive')

    # Move Simulations to avoid cluttering the drive folder
    # if not os.path.exists('/content/simulations'):
    #   os.mkdir('/content/simulations')

    # if list(os.listdir('/content/simulations')) == []:
    #   copy_tree('/content/drive/MyDrive/TFM/simulations', '/content/simulations')

    # unzip all simulations
    # print("Name of the already simulated data: \n", )
    for zip_file in os.listdir('/content/simulations'):
        if zip_file.endswith('.zip'):
            print(" ----> " + zip_file)
            with zipfile.ZipFile("/content/simulations/" + zip_file, 'r') as zip_ref:
                zip_ref.extractall('/content/simulations/')

    SIMULATIONS_PATH: str = "/content/simulations"
else:
    if not os.path.exists('./data/simulations'): os.mkdir('./data/simulations')
    for zip_file in os.listdir('data'):
        if zip_file.endswith('.zip'):
            print(" ----> " + zip_file)
            with zipfile.ZipFile("./data/" + zip_file, 'r') as zip_ref:
                zip_ref.extractall('./data/simulations')
    SIMULATIONS_PATH: str = "./data/simulations"

In [4]:
cmg   = np.load(SIMULATIONS_PATH + '/Channel_matrix_gain.npy')
alloc = np.load(SIMULATIONS_PATH + '/sisa-allocation.npy')

# get sample from all
n_sample = 12_000
cmg   = cmg[:n_sample]
alloc = alloc[:n_sample]

n_sample = cmg.shape[0]
K, N, _  = cmg.shape[1:]

shape    = lambda s: " x".join([f"{d:3d}" for d in s])
print(f"channel    matrix shape: {shape(cmg.shape)} \nallocation matrix shape: {shape(alloc.shape)}")

channel    matrix shape: 12000 x  4 x 20 x 20 
allocation matrix shape: 12000 x 20


## Publications to revise

* (power) Power control for 6g industrial wireless subnetworks: A graph neural network approach
* (allocation) Towards 6g in-x subnetworks with sub-millisecond communication cycles and extreme reliability
* (power) Multi-agent deep reinforcement learning for dynamic power allocation in wireless networks
* (both) Multi-agent reinforcement learning for dynamic resource management in 6g in-x subnetworks
* (both) Multi-agent dynamic resource allocation in 6g in-x subnetworks with limited sensing information

## First Proposal

In this proposal, we could mixed different implementations for optimization of problem.
We can only consider the following setup:

1. Determine a almost optimal subband allocation for the networks. We could use a power selection of $p = p_{max}$
2. Based on the obtained allocation, we determine a power control for each subnetwork that minimizes the used 
power and does not deteriorite the signal.

For the subband allocation, we could consider the implementation from this [publication](https://ieeexplore.ieee.org/document/10597067).


In [8]:
# First Step: Subband allocation problem
class RateConfirmAllocModel(nn.Module):
    def __init__(self, n_subnetworks: int, n_bands: int, hidden_dim: int = 1000, hidden_layers: int = 4) -> None:
        super().__init__()

        # initialize state
        self.n = n_subnetworks
        self.k = n_bands

        # DNN architecture
        self.input_size = self.n * self.n
        self.output_size = self.n * self.k
        
        layers = [nn.BatchNorm1d(self.input_size)]
        dims = [self.input_size] + [hidden_dim] * (hidden_layers - 1) + [self.output_size]
        for i in range(1, hidden_layers + 1):
            # linear layers with HE initialization
            layers.append(nn.Linear(dims[i - 1], dims[i]))
            torch.nn.init.kaiming_normal_(layers[-1].weight, nonlinearity='relu')     
            
            # apply dropout. We have a lot of parameters, it is required
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.05))
            layers.append(nn.BatchNorm1d(dims[i]))

        layers = layers[:-3]
        self.model = nn.Sequential(*layers)

    @staticmethod
    def preprocess(channel_gain: np.ndarray | torch.Tensor ) -> torch.Tensor:
        if len(channel_gain.shape[1:]) == 3: 
            channel_gain = torch.mean(channel_gain, dim = 1)
        
        channel_gain = torch.tensor(channel_gain, requires_grad=False)
        channel_gain = 10 * torch.log10(channel_gain) # transform to Dbm
        return channel_gain.type(torch.float32)

    def forward(self, channel_gain: torch.Tensor ) -> torch.Tensor:
        # preprocess to obtain a NxN channel gain
        channel_gain = self.preprocess(channel_gain)
        # flatten the inputs
        flattened_channel = channel_gain.reshape(-1, self.input_size)
        # apply model
        channel_network = self.model(flattened_channel)
        # determine best allocation
        channel_network = channel_network.reshape(-1, self.k, self.n)
        # derive probabilities
        return F.softmax(channel_network, dim = 1)

The used loss function in the first-stage correspond to the 

In [9]:
def shannon_rate(C: torch.Tensor, A: torch.Tensor, config: SimConfig):
    B, K, N, _ = C.shape
    NE         = config.noise_power
    P          = config.transmit_power

    # calculate signal and interference with probabilistic allocations
    ids    = range(N)
    signal = C[:, :, ids, ids] * A[:, :, :] * P
    interference = torch.sum((C[:, :, :, :] * A[:, :, :, None]) * P, dim=2) - signal
    interference = interference + NE + 1e-9
    
    # compute individual rate
    sinr = signal / interference
    rate = torch.log2(1 + sinr)
    rate = torch.sum(rate, dim = 1)
    return rate

def loss_fullfield_req(config: SimConfig, channel_gain: torch.Tensor, allocation: torch.Tensor, req: float) -> torch.Tensor:
    rate = shannon_rate(channel_gain, allocation, config)
    rate = F.sigmoid(req - rate)
    return torch.mean(rate / req, dim = 1)


In [ ]:
BATCH_SIZE: int = 1024
MAX_EPOCH : int = 200

REQ: float      = 5.

# build datasets
TRAIN_SAMPLE: int = 10_000
VALID_SAMPLE: int =  5_000
TESTS_SAMPLE: int =  5_000

train_data = TensorDataset(torch.tensor(cmg[:TRAIN_SAMPLE]))
train_data = DataLoader(train_data, batch_size=BATCH_SIZE)

valid_data = TensorDataset(torch.tensor(cmg[TRAIN_SAMPLE:TRAIN_SAMPLE + VALID_SAMPLE]))
valid_data = DataLoader(valid_data, batch_size=BATCH_SIZE)

tests_data = TensorDataset(torch.tensor(cmg[TRAIN_SAMPLE+VALID_SAMPLE:TRAIN_SAMPLE + VALID_SAMPLE + TESTS_SAMPLE]))
tests_data = DataLoader(tests_data, batch_size=BATCH_SIZE)

# training config
LR: float  = 0.01


name  = "p1-alloc-dnn-00-01-base"
learning_config = {
    'batch-size': BATCH_SIZE,
    'n_epochs'  : MAX_EPOCH,
    'lr'        : LR,
    'dropout'   : 0.01,
    'data-split': f"{TRAIN_SAMPLE}-{VALID_SAMPLE}-{TESTS_SAMPLE}",
    'network-req-nbit': f'{REQ:3.2f}',
    'network-req-snir': f'{2 ** (REQ) - 1}'
}

def binarization_error(alloc: torch.Tensor) -> float:
    rounded = torch.round(alloc)
    return torch.mean(torch.abs(alloc - rounded))

def update_metrics(metrics, alloc, sample):
    alloc = torch.argmax(alloc.detach(), dim = 1)

    batch_size = sample.size(0)
    for b in range(batch_size):
        allocation  = alloc[b].detach().cpu().numpy()
        gain_matrix = sample[b].detach().cpu().numpy()

        rates       = rate_metrics.bit_rate(config, gain_matrix, allocation, config.transmit_power)
        fairness    = rate_metrics.jain_fairness(rates)
        spectral    = np.mean(rate_metrics.spectral_efficency(config, rates))
        rates       = np.mean(rates)

    metrics['bit-rate'] += rates / batch_size
    metrics['jain-fairness'] += fairness / batch_size
    metrics['spectral-efficency'] += spectral / batch_size
    return metrics

# setup_wandb(name, learning_config)
model = RateConfirmAllocModel(20, 4).to(device)
optimizer = optim.Adam(model.parameters(), LR)
for epoch in trange(MAX_EPOCH, desc = "training epoch", unit = "epoch"):
    # training step
    model.train()
    training_loss = 0.
    train_binary_loss = 0.

    training_metrics = {
        'bit-rate': 0.,
        'jain-fairness': 0.,
        'spectral-efficency': 0.
    }

    for sample in tqdm(train_data, desc = 'training step:', unit = 'batch', total = len(train_data), leave=False):
        optimizer.zero_grad()

        sample     = sample[0]
        alloc_prob = model(sample.to(device))
        loss       = loss_fullfield_req(config, sample, alloc_prob, REQ).mean()
        training_loss += loss.item()

        loss.backward()
        optimizer.step()

        train_binary_loss += binarization_error(alloc_prob.detach())

        # training_metrics = update_metrics(training_metrics, alloc_prob, sample)

    training_loss = training_loss / len(train_data)
    train_binary_loss = train_binary_loss / len(train_data)
    training_metrics = { 'train-' + key: val / len(train_data) for key, val in training_metrics.items()}

    model.eval()
    validation_loss = 0.
    valid_binary_loss = 0.
    validation_metrics = {
        'bit-rate': 0.,
        'jain-fairness': 0.,
        'spectral-efficency': 0.
    }
    for sample in tqdm(valid_data, desc = 'validation step:', unit = 'batch', total = len(valid_data), leave = False):
        optimizer.zero_grad()
        
        sample     = sample[0]
        alloc_prob = model(sample)
        loss       = loss_fullfield_req(config, sample, alloc_prob, REQ).mean()
        validation_loss += loss.item()

        # validation_metrics = update_metrics(validation_metrics, alloc_prob, sample)

    vaidation_loss = validation_loss / len(train_data)
    valid_binary_loss = valid_binary_loss / len(valid_data)

    validation_metrics = { 'valid-' + key: val / len(valid_data) for key, val in validation_metrics.items()}
    
    logged_values = {
        'train-loss': training_loss, 'valid-loss': validation_loss, 
        'train-binary-loss': train_binary_loss, 'valid-binary-loss': valid_binary_loss
    }

    logged_values.update(training_metrics)
    logged_values.update(validation_metrics)
    print(logged_values)
    # wandb.log(logged_values)

wandb.finish()


training epoch:   0%|          | 0/2 [00:00<?, ?epoch/s]

training step::   0%|          | 0/10 [00:00<?, ?batch/s]

validation step::   0%|          | 0/2 [00:00<?, ?batch/s]

{'train-loss': 0.07050831734227296, 'valid-loss': 0.2978861067858164, 'train-binary-loss': tensor(0.1893), 'valid-binary-loss': 0.0, 'train-bit-rate': 0.0, 'train-jain-fairness': 0.0, 'train-spectral-efficency': 0.0, 'valid-bit-rate': 0.0, 'valid-jain-fairness': 0.0, 'valid-spectral-efficency': 0.0}


training step::   0%|          | 0/10 [00:00<?, ?batch/s]

validation step::   0%|          | 0/2 [00:00<?, ?batch/s]

{'train-loss': 0.05494073937393671, 'valid-loss': 0.27588836982328757, 'train-binary-loss': tensor(0.2310), 'valid-binary-loss': 0.0, 'train-bit-rate': 0.0, 'train-jain-fairness': 0.0, 'train-spectral-efficency': 0.0, 'valid-bit-rate': 0.0, 'valid-jain-fairness': 0.0, 'valid-spectral-efficency': 0.0}
